In [6]:
!pip -qqq install pip --progress-bar off
!pip -qqq install langchain-groq==0.1.3 --progress-bar off
!pip -qqq install langchain==0.1.17 --progress-bar off
!pip -qqq install llama-parse==0.1.3 --progress-bar off
!pip -qqq install qdrant-client==1.9.1  --progress-bar off
!pip -qqq install "unstructured[md]"==0.13.6 --progress-bar off
!pip -qqq install fastembed==0.2.7 --progress-bar off
!pip -qqq install flashrank==0.2.4 --progress-bar off
!pip install groq --progress-bar off
!pip install PyMuPDF --progress-bar off


ERROR: Could not find a version that satisfies the requirement onnxruntime<2.0.0,>=1.17.0 (from fastembed) (from versions: 1.12.0, 1.12.1, 1.13.1, 1.14.0, 1.14.1, 1.15.0, 1.15.1, 1.16.0, 1.16.1, 1.16.2, 1.16.3)
ERROR: No matching distribution found for onnxruntime<2.0.0,>=1.17.0


In [9]:
import os
import textwrap
# from google.colab import userdata
from IPython.display import Markdown
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_community.embeddings.fastembed import FastEmbedEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_groq import ChatGroq
from llama_parse import LlamaParse


In [30]:
import boto3
import fitz  # PyMuPDF

# Specify the S3 bucket and file name
s3_bucket = 'constitutionnepal'
s3_key = 'constitutionnepal/Constitution_Nepal.pdf'
local_filename = 'Constitution_Nepal.pdf'

# Initialize a session using Amazon S3
s3 = boto3.client('s3')

# Download the file from S3
s3.download_file(s3_bucket, s3_key, local_filename)

# # Open the PDF file using PyMuPDF
# document = fitz.open(local_filename)


# page = document.load_page(0)
# text = page.get_text()

# print(text)

# document.close()


In [22]:
os.environ["GROQ_API_KEY"] = ""

# def print_response(response):

def print_response(response):
    response_txt = response["result"]
    for chunk in response_txt.split("\n"):
        if not chunk:
            print()
            continue
        print("\n".join(textwrap.wrap(chunk, 100, break_long_words=False)))



# Document parsing

In [23]:
#Document parsing (cleaning the document)
instruction = """The provided document is the Constitution of Nepal 2072. This document encompasses all the legal frameworks, guidelines, and principles governing the country. It defines the structure of the state, the distribution of powers between different levels of government, and the fundamental rights and duties of citizens.

When answering questions based on this document, please follow these guidelines:
- Be precise and concise in your responses.
- Ensure that the information is accurate and directly relevant to the question.
- Highlight key articles, sections, or provisions where applicable.
- Provide context to your answers when necessary, explaining the implications or importance of specific provisions.
- Maintain a neutral and informative tone, avoiding any personal opinions or interpretations.

The goal is to provide clear and informative answers that help the user understand the specific aspects of the Constitution of Nepal 2072."""

parser = LlamaParse(
    api_key = '',
    result_type = 'markdown',
    parsing_instruction = instruction,
    max_timeout = 5000,

)

llama_parse_document = await parser.aload_data(local_filename)

Started parsing the file under job_id cac11eca-e988-4f36-b9fa-8062fec85d7b


In [24]:
parsed_doc = llama_parse_document[0]
Markdown(parsed_doc.text[:1000])

# The Constitution of Nepal

|Date of Publication in Nepal Gazette|20 September 2015 (2072.6.3)|
|---|---|
|The Constitution of Nepal (First Amendment), 2016 (2072)|28 February 2016 (2072.11.16)|

Preamble:

We, the Sovereign People of Nepal, Internalizing the people's sovereign right and right to autonomy and self-rule, while maintaining freedom, sovereignty, territorial integrity, national unity, independence and dignity of Nepal, Recalling the glorious history of historic people's movements, armed conflict, dedication and sacrifice undertaken by the Nepalese people at times for the interest of the nation, democracy and progressive changes, and respecting for the martyrs and disappeared and victim citizens, Ending all forms of discrimination and oppression created by the feudalistic, autocratic, centralized, unitary system of governance, Protecting and promoting social and cultural solidarity, tolerance and harmony, and unity in diversity by recognizing the multi-ethnic, multi-lingua

In [36]:
output_file_key = 'constitutionnepal/Processed_Constitution_Nepal.md'
local_output_file_name = 'Processed_Constitution_Nepal.md'

# Save the cleaned text to a new PDF (assuming you have a way to convert text to PDF)
with open(local_output_file_name, 'w') as file:
    file.write(parsed_doc.text)

# Upload the cleaned file back to S3
s3.upload_file(local_output_file_name, bucket_name, output_file_key)

print(f'Processed file has been uploaded to s3://{bucket_name}/{output_file_key}')

Processed file has been uploaded to s3://constitutionnepal/constitutionnepal/Processed_Constitution_Nepal.md


# Vector embedding

In [37]:
# Handle markdown file
loader = UnstructuredMarkdownLoader(local_output_file_name)
loaded_documents = loader.load()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [38]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=2048, chunk_overlap=128)
docs = text_splitter.split_documents(loaded_documents)
print(docs[0].page_content)

The Constitution of Nepal

Date of Publication in Nepal Gazette 20 September 2015 (2072.6.3) The Constitution of Nepal (First Amendment), 2016 (2072) 28 February 2016 (2072.11.16)

Preamble:

We, the Sovereign People of Nepal, Internalizing the people's sovereign right and right to autonomy and self-rule, while maintaining freedom, sovereignty, territorial integrity, national unity, independence and dignity of Nepal, Recalling the glorious history of historic people's movements, armed conflict, dedication and sacrifice undertaken by the Nepalese people at times for the interest of the nation, democracy and progressive changes, and respecting for the martyrs and disappeared and victim citizens, Ending all forms of discrimination and oppression created by the feudalistic, autocratic, centralized, unitary system of governance, Protecting and promoting social and cultural solidarity, tolerance and harmony, and unity in diversity by recognizing the multi-ethnic, multi-lingual, multi-religio

In [41]:
!pip install fastembed

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 kB 17.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached loguru-0.7.2-py3-none-any.whl.metadata (23 kB)
  Using cached onnx-1.16.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (16 kB)
INFO: pip is looking at multiple versions of fastembed to determine which version is compatible with other requirements. This could take a while.
  Using cached fastembed-0.2.7-py3-none-any.whl.metadata (5.3 kB)
  Using cached huggingface_hub-0.20.3-py3-none-any.whl.metadata (12 kB)
INFO: pip is still looking at multiple versions of fastembed to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 4.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.9/15.9 MB 25.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 26.2 MB/s eta 0:00:0000:0100:01
  Attempting u

In [1]:
import boto3
import shutil
import os

# output_file_key = 'constitutionnepal/qdrant_db'
# local_output_file_name = 'qdrant_db'
# Define the S3 bucket and file key

output_file_key = 'constitutionnepal/qdrant_db.zip'
local_output_file_name = 'qdrant_db.zip'


# Model embedding
embeddings = FastEmbedEmbeddings(
    model_name = "BAAI/bge-base-en-v1.5")

# Create the Qdrant database from documents
qdrant = Qdrant.from_documents(
    docs,
    embeddings,
    # path=local_db_path,
    collection_name="document_embeddings"
)

# Compress the Qdrant database directory
shutil.make_archive(local_db_path, 'zip', local_db_path)

# Create an S3 client
s3 = boto3.client('s3')

# Upload the compressed Qdrant database to S3
s3.upload_file(local_output_file_name, bucket_name, output_file_key)

print(f'Qdrant database has been uploaded to s3://{bucket_name}/{output_file_key}')


NameError: name 'FastEmbedEmbeddings' is not defined

In [12]:
%%time
query = "What fundamental rights are guaranteed to Nepali citizens?"
similar_docs = qdrant.similarity_search_with_score(query)

CPU times: user 306 ms, sys: 5.99 ms, total: 312 ms
Wall time: 314 ms


In [13]:
for doc, score in similar_docs:
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {score}")
    print("_" * 80)
    print()

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 

score: 0.7861023710448288
________________________________________________________________________________

text: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law, be inviolable.

Right against exploitation

Every person shall ha

score: 0.780478909497755
________________________________________________________________________________

text: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law, be inviolable.

Right against exploitation

Every person sha

In [14]:
%%time
retriever = qdrant.as_retriever(search_kwargs = {"k": 5})
retrieved_docs = retriever.invoke(query)

CPU times: user 348 ms, sys: 60.8 ms, total: 409 ms
Wall time: 348 ms


In [15]:
for doc in retrieved_docs:
  print(f"id: {doc.metadata['_id']}\n")
  print(f"text: {doc.page_content[:256]}")
  print("_" * 80)
  print()

id: 79c0a32a4165458daee885f410c948dc

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party shall have the right to free legal aid in accordance with law.

Right of victim of crime:

(1) A victim of crime shall 
________________________________________________________________________________

id: eb22328c91b14ad098533156e58de6e6

text: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law, be inviolable.

Right against exploitation

Every person shall ha
________________________________________________________________________________

id: b25ab55d3c00447a96a11f805c0cf23d

text: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law,

In [16]:
# path = '/content/drive/MyDrive/LLM-Nepal-constitution'

### Reranking

In [17]:
# Rerank and compress
compressor = FlashrankRerank(model = 'ms-marco-MiniLM-L-12-v2')
compression_retriever = ContextualCompressionRetriever(
    base_compressor = compressor,
    base_retriever = retriever)

ms-marco-MiniLM-L-12-v2.zip: 100%|██████████| 21.6M/21.6M [00:00<00:00, 61.9MiB/s]


In [18]:
%%time
reranked_docs = compression_retriever.invoke(query)
len(reranked_docs)

Running pairwise ranking..
CPU times: user 3.02 s, sys: 158 ms, total: 3.18 s
Wall time: 3.27 s


3

In [19]:
for doc in reranked_docs:
    print(f"id: {doc.metadata['_id']}\n")
    print(f"text: {doc.page_content[:256]}\n")
    print(f"score: {doc.metadata['relevance_score']}")
    print("-" * 80)
    print()

id: eb22328c91b14ad098533156e58de6e6

text: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law, be inviolable.

Right against exploitation

Every person shall ha

score: 0.8946086168289185
--------------------------------------------------------------------------------

id: b25ab55d3c00447a96a11f805c0cf23d

text: Right to privacy

The privacy of any person, his or her residence, property, document, data, correspondence and matters relating to his or her character shall, except in accordance with law, be inviolable.

Right against exploitation

Every person shall ha

score: 0.8946086168289185
--------------------------------------------------------------------------------

id: 79c0a32a4165458daee885f410c948dc

text: Every person shall have the right to a fair trial by an independent, impartial and competent court or judicial body.

Any indigent party

In [20]:
llm = ChatGroq(temperature = 0, model = 'llama3-70b-8192')

In [21]:
prompt_template = """
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Answer the question and provide additional helpful information,
based on the pieces of information, if applicable. Be succinct.

Responses should be properly formatted to be easily read.
"""

prompt = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)

In [22]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose" : True},
)

In [23]:
%%time

response = qa.invoke("what are the fundamental rights guaranteed to Nepali citizens?")

Running pairwise ranking..


> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: Other provisions relating to citizenship of Nepal:

Other matters relating to the maintenance of records setting out the identity of every citizen of Nepal and the citizenship of Nepal shall be as provided for in the Federal law.

Part-3

Fundamental Rights and Duties

Right to live with dignity:

Every person shall have the right to live with dignity.

No law shall be made providing for the death penalty to any one.

Right to freedom:

No person shall be deprived of his or her personal liberty except in accordance with law.

Every citizen shall have the following freedoms:

(a) freedom of opinion and expression,

(b) freedom to assemble peaceably and without arms,

(c) freedom to for

In [24]:
print_response(response)

**Fundamental Rights Guaranteed to Nepali Citizens:**

1. **Right to Live with Dignity**: Every person has the right to live with dignity, and no law shall
provide for the death penalty.
2. **Right to Freedom**: Every citizen has the freedom of opinion and expression, assembly, forming
political parties, unions, and associations, moving and residing in any part of Nepal, and
practicing any profession.
3. **Right to Privacy**: The privacy of every person, their residence, property, document, data,
correspondence, and matters relating to their character shall be inviolable, except in accordance
with law.
4. **Right Against Exploitation**: Every person has the right against exploitation on any grounds,
and no one shall be subjected to trafficking, slavery, or servitude, or forced to work against their
will.
5. **Right to Clean Environment**: Every citizen has the right to live in a clean and healthy
environment, and the victim has the right to obtain compensation for any injury caused by


In [25]:
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": prompt, "verbose" : False},
)

In [26]:
%%time

response = qa.invoke("what are the fundamental rights guaranteed to Nepali citizens?")

Running pairwise ranking..
CPU times: user 2.09 s, sys: 146 ms, total: 2.23 s
Wall time: 3.33 s


In [27]:
print_response(response)

**Fundamental Rights Guaranteed to Nepali Citizens:**

1. **Right to Live with Dignity**: Every person has the right to live with dignity, and no law shall
provide for the death penalty.
2. **Right to Freedom**: Every citizen has the freedom of opinion and expression, assembly, forming
political parties, unions, and associations, moving and residing in any part of Nepal, and
practicing any profession.
3. **Right to Privacy**: The privacy of every person, their residence, property, document, data,
correspondence, and matters relating to their character shall be inviolable, except in accordance
with law.
4. **Right Against Exploitation**: Every person has the right against exploitation on any grounds,
and no one shall be subjected to trafficking, slavery, or servitude, or forced to work against their
will.
5. **Right to Clean Environment**: Every citizen has the right to live in a clean and healthy
environment, and the victim has the right to obtain compensation for any injury caused by


In [28]:
Markdown(response["result"])

**Fundamental Rights Guaranteed to Nepali Citizens:**

1. **Right to Live with Dignity**: Every person has the right to live with dignity, and no law shall provide for the death penalty.
2. **Right to Freedom**: Every citizen has the freedom of opinion and expression, assembly, forming political parties, unions, and associations, moving and residing in any part of Nepal, and practicing any profession.
3. **Right to Privacy**: The privacy of every person, their residence, property, document, data, correspondence, and matters relating to their character shall be inviolable, except in accordance with law.
4. **Right Against Exploitation**: Every person has the right against exploitation on any grounds, and no one shall be subjected to trafficking, slavery, or servitude, or forced to work against their will.
5. **Right to Clean Environment**: Every citizen has the right to live in a clean and healthy environment, and the victim has the right to obtain compensation for any injury caused by environmental pollution or degradation.
6. **Right to Education**: Every citizen has the right to access basic education, compulsory and free education up to the basic level, and free education up to the secondary level from the State. Additionally, citizens with disabilities and economically indigent citizens have the right to free higher education, and every Nepalese community has the right to education in their mother tongue.
7. **Right to Language and Culture**: (No specific details provided in the given context)

These fundamental rights are guaranteed to Nepali citizens, ensuring their dignity, freedom, privacy, and access to education, among other essential aspects of life.

In [29]:
%%time
response = qa.invoke("What are the powers and responsibilities of federal, provincial, and local governments?")
Markdown(response["result"])



Running pairwise ranking..
CPU times: user 1.89 s, sys: 131 ms, total: 2.02 s
Wall time: 3.4 s


**Powers and Responsibilities of Federal, Provincial, and Local Governments**

The provided information only mentions the powers and responsibilities of the Local Level government. It does not provide information on the powers and responsibilities of the Federal and Provincial governments.

**Local Level Government:**

The executive power of the Local Level is vested in the Village Executive or the Municipal Executive. The local executive power includes:

* Issuing general directives, controlling, and regulating the governance of the Village Body and the Municipality
* Performing executive functions of the Village Body and the Municipality
* Authenticating decisions or orders issued in the name of the Village Executive and the Municipal Executive

The Village Executive consists of a Chairperson, Vice-Chairperson, Ward Chairperson elected from each Ward, and members elected pursuant to clause (4). The Chairperson and Vice-Chairperson are elected by the voters residing within the concerned Village Body area by secret ballots.

The Local Level government is responsible for various matters, including:

* Town police
* Cooperative institutions
* Operation of F.M.
* Local taxes and fees
* Management of local services
* Collection of local statistics and records
* Local level development plans and projects
* Basic and secondary education
* Basic health and sanitation
* Local market management, environment protection, and bio-diversity
* Local roads, rural roads, agro-roads, irrigation
* Management of Village Assembly, Municipal Assembly, District Assembly, local courts, mediation, and arbitration
* Local government services, including local records management, distribution of house and land ownership certificates, agriculture and animal husbandry, management of senior citizens, persons with disabilities, and incapacitated, collection of statistics of the unemployed, management, operation, and control of agricultural extension, water supply, small hydropower projects, alternative energy, disaster management, protection of watersheds, wildlife, mines, and minerals, and protection and development of languages, cultures, and fine arts.

Note: The information provided does not mention the powers and responsibilities of the Federal and Provincial governments.

In [30]:
%%time
response = qa.invoke("Who is Binit")
Markdown(response["result"])


Running pairwise ranking..
CPU times: user 3.33 s, sys: 59.6 ms, total: 3.39 s
Wall time: 3.93 s


I don't know who Binit is. The provided information does not mention a person named Binit. It appears to be a set of rules and regulations related to the State Government, Chief Minister, and Ministers, but it does not contain any information about a person named Binit.

In [31]:
%%time
response = qa.invoke("how has the constitution of nepal divided the revenue under federalism?")
Markdown(response["result"])


Running pairwise ranking..
CPU times: user 1.67 s, sys: 112 ms, total: 1.78 s
Wall time: 2.82 s


**Answer:** The Constitution of Nepal has divided the revenue under federalism in a way that:

* The Federation, State, and Local level may impose taxes on matters falling within their respective jurisdiction and collect revenue from these sources.
* The Government of Nepal shall make provisions for the equitable distribution of the collected revenue to the Federation, State, and Local level.
* The amount of fiscal transfer receivable by the State and Local level shall be as recommended by the National Natural Resources and Fiscal Commission.
* The Government of Nepal shall distribute fiscal equalization grants to the State and Local level based on the need of expenditure and revenue capacity.
* Each State shall distribute fiscal equalization grants to its subordinate Local level based on the need of expenditure and revenue capacity.
* Provisions relating to distribution of conditional grants, complementary grants, or special grants for other purposes shall be as provided for in the Federal law.

**Additional helpful information:**

* The distribution of revenues between the Federal, State, and Local level shall be made in a balanced and transparent manner.
* The Government of Nepal has the authority to determine provisions relating to the imposition of taxes and collection of revenue on matters that fall within the Concurrent List and on matters that are not included in the List of any level.

In [32]:
%%time
response = qa.invoke("list out the directive principles of state under constitution of nepal?")
Markdown(response["result"])


Running pairwise ranking..
CPU times: user 2.86 s, sys: 78.6 ms, total: 2.94 s
Wall time: 4.35 s


Based on the provided information, the directive principles of the State under the Constitution of Nepal are:

1. Establishing a public welfare system of governance
2. Establishing a just system in all aspects of national life through:
	* Rule of law
	* Values and norms of fundamental rights and human rights
	* Gender equality
	* Proportional inclusion
	* Participation
	* Social justice
3. Protecting the life, property, equality, and liberties of the people
4. Consolidating a federal democratic republican system of governance
5. Ensuring an atmosphere conducive to the enjoyment of the fruits of democracy
6. Maintaining relations between Federal Units on the basis of cooperative federalism
7. Incorporating the principle of proportional participation in the system of governance on the basis of local autonomy and decentralization
8. Building a civilized and egalitarian society by eliminating all forms of discrimination, exploitation, and injustice
9. Developing social and cultural values founded on:
	* National pride
	* Democracy
	* Pro-people
	* Respect of labor
	* Entrepreneurship
	* Discipline
	* Dignity
	* Harmony
10. Consolidating national unity by maintaining social cohesion, solidarity, and harmony while recognizing cultural diversity.

Additional helpful information:

* These directive principles are guiding principles for the governance of the State (Part-4, Article 49).
* The State shall mobilize means and resources to implement these principles (Part-4, Article 49).
* The Government of Nepal shall submit an annual report on the implementation of these principles to the President, who shall lay it before the Federal Parliament (Part-5, Article 53).
* A committee in the Federal Parliament shall monitor and evaluate the implementation of these principles (Part-5, Article 54).

In [14]:
print('hello')

hello
